In [1]:
import numpy as np
import pandas as pd

In [2]:
read_csv = pd.read_csv("/Users/jooh8/Documents/GitHub/Quant/data/A000660/Index_A000660.txt", sep=" ", low_memory=False)

In [3]:
len(read_csv)

5615

In [4]:
for period in range(10):
    
    read_csv["고가_{}".format(period)] = read_csv["고가"].shift(periods=period).fillna(0)

In [5]:
for period in range(10):
    
    read_csv["최고가_{}".format(period)] = read_csv[["고가_{}".format(i) for i in range(period + 1)]].apply(np.max, 1)

In [6]:
for period in range(10):
    
    read_csv["Target_{}".format(period)] = read_csv["시가"] * 1.05 < read_csv["고가_{}".format(period)]

In [7]:
columns = list(read_csv.columns)

In [8]:
momentums = columns[11:-30]
print(momentums)

['A Ratio_20', 'B Ratio_20', 'Aroon_20 Up', 'Down', 'Aroon Osillator_20', 'ATR_14', 'BB-RSI_종가_10', 'BB-RSI_20_2.00 상한', 'BB-RSI 하한', 'BPDL RSI_14', 'BPDL Stochatic_14', 'Chande Momentum Oscillator_10', "Chaikin's Volatility_14,14", 'CompuTrac Volatility_10', '+DI_14', '-DI_14', 'ADX_14', 'ADXR_14', '+DI(simple)_14', '-DI(simple)_14', 'ADX(simple)_14,14', 'ADXR(simple)_14,14,14', 'Energy+_14', 'Energy-_14', 'High Low Envelope_1', 'High Low Oscillator_3', 'Inertia_20,14,10', 'Klinger Oscillator', 'Linear Trend Oscillator_10,20', 'LRS_종가,14', 'Mass Index_20,9', 'Morris Mixed Momentum', 'New BPDL HiLo Index MA_14,7', 'Open Difference_15', 'QStick_14', 'Random Walk Index_15,3', 'RCI_5', 'RCI_9', 'RCI_13', 'RCI_18', 'Relative Volatility Index_단순,14,10', 'Reverse 단기_12', ' 장기_24', 'RSI_종가,14', 'RSI(simple)_종가,14', 'Sigma_종가,20', 'SMI_5,3,3', 'Standard Deviation_14,2.00', 'Standard Error_종가,14', 'Fast %K_5', 'Fast %D_3', 'Slow %K_5,3', 'Slow %D_3', 'Slow(Simple) %K_5,3', 'Slow(Simple) %D_3', 

In [9]:
targets = columns[-10:]
print(targets)

['Target_0', 'Target_1', 'Target_2', 'Target_3', 'Target_4', 'Target_5', 'Target_6', 'Target_7', 'Target_8', 'Target_9']


In [10]:
X_train = read_csv[-2000:-100][momentums]
Y_train = read_csv[-2000:-100][targets]

In [11]:
Y_train.apply(np.sum, 0)

Target_0     32
Target_1     71
Target_2    188
Target_3    281
Target_4    344
Target_5    365
Target_6    401
Target_7    407
Target_8    436
Target_9    456
dtype: int64

In [12]:
X_test = read_csv[-100:][momentums]
Y_test = read_csv[-100:][targets]

In [13]:
Y_test.apply(np.sum, 0)

Target_0     2
Target_1     2
Target_2    11
Target_3    17
Target_4    24
Target_5    21
Target_6    21
Target_7    23
Target_8    21
Target_9    27
dtype: int64

In [14]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import numpy as np

In [15]:
scaler = preprocessing.StandardScaler().fit(X_train)

C:\Users\jooh8\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [16]:
forest = RandomForestClassifier(n_estimators=100, random_state=1, max_depth=5)

In [17]:
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

In [18]:
multi_target_forest.fit(scaler.transform(X_train), Y_train)

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=-1)

In [19]:
predict = multi_target_forest.predict(scaler.transform(X_train))

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [20]:
corr = predict == Y_train

In [21]:
corr.apply(np.mean, 0)

Target_0    0.993158
Target_1    0.973684
Target_2    0.941053
Target_3    0.933158
Target_4    0.934211
Target_5    0.941053
Target_6    0.935789
Target_7    0.952105
Target_8    0.951053
Target_9    0.952632
dtype: float64

In [22]:
np.mean(corr.apply(np.mean, 0))

0.9507894736842106

In [23]:
"""
Target_0    0.997895
Target_1    0.990000
Target_2    0.964211
Target_3    0.944737
Target_4    0.931579
Target_5    0.931579
Target_6    0.927895
Target_7    0.933684
Target_8    0.935789
Target_9    0.933684
dtype: float64
0.9491052631578947
"""

'\nTarget_0    0.997895\nTarget_1    0.990000\nTarget_2    0.964211\nTarget_3    0.944737\nTarget_4    0.931579\nTarget_5    0.931579\nTarget_6    0.927895\nTarget_7    0.933684\nTarget_8    0.935789\nTarget_9    0.933684\ndtype: float64\n0.9491052631578947\n'

In [24]:
predict = multi_target_forest.predict(scaler.transform(X_test))

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [25]:
corr = predict == Y_test

In [26]:
corr.apply(np.mean, 0)

Target_0    0.98
Target_1    0.98
Target_2    0.87
Target_3    0.96
Target_4    0.89
Target_5    0.90
Target_6    0.90
Target_7    0.95
Target_8    0.96
Target_9    0.95
dtype: float64

In [27]:
np.mean(corr.apply(np.mean, 0))

0.9339999999999999

In [28]:
"""
Target_0    1.00
Target_1    1.00
Target_2    0.95
Target_3    0.89
Target_4    0.91
Target_5    0.92
Target_6    0.95
Target_7    0.95
Target_8    0.92
Target_9    0.92
dtype: float64
0.9410000000000001
"""

'\nTarget_0    1.00\nTarget_1    1.00\nTarget_2    0.95\nTarget_3    0.89\nTarget_4    0.91\nTarget_5    0.92\nTarget_6    0.95\nTarget_7    0.95\nTarget_8    0.92\nTarget_9    0.92\ndtype: float64\n0.9410000000000001\n'

In [31]:
score = multi_target_forest.predict_proba(scaler.transform(X_test))

C:\Users\jooh8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [33]:
score.shape

AttributeError: 'list' object has no attribute 'shape'

In [34]:
len(score)

[array([[0.9755708 , 0.0244292 ],
        [0.994028  , 0.005972  ],
        [0.99729644, 0.00270356],
        [0.99693247, 0.00306753],
        [0.98250263, 0.01749737],
        [0.86059274, 0.13940726],
        [0.94238298, 0.05761702],
        [0.94853433, 0.05146567],
        [0.90148457, 0.09851543],
        [0.95595882, 0.04404118],
        [0.98923198, 0.01076802],
        [0.95012624, 0.04987376],
        [0.97275395, 0.02724605],
        [0.99348845, 0.00651155],
        [0.89938186, 0.10061814],
        [0.56002344, 0.43997656],
        [0.8481694 , 0.1518306 ],
        [0.93745017, 0.06254983],
        [0.95251656, 0.04748344],
        [0.94945917, 0.05054083],
        [0.96083362, 0.03916638],
        [0.95455325, 0.04544675],
        [0.9547915 , 0.0452085 ],
        [0.94439729, 0.05560271],
        [0.99424497, 0.00575503],
        [0.99528378, 0.00471622],
        [0.88520418, 0.11479582],
        [0.86172357, 0.13827643],
        [0.95561526, 0.04438474],
        [0.977